One Shot Learning - Facial Recognition

We will use Siamese Network, which does not require huge data to train.

Siamese Network consists of two Symmetrical Neural Networks having same weights and architectures. And at last they are joined together (at end ) which acts as distance function.

FaceNet by Google : https://arxiv.org/abs/1503.03832

Idea here is that, Both the networks that return embedding vectors. 
 Two embeddings belonging to the same person should have a lower distance, while embeddings belonging to different person’s image should show a larger distance.

 

FaceNet Network

Here we are using encodings of anchor, positive, negative images of shape (None, 128) vector

In [0]:
from keras import backend as K
import time
from multiprocessing.dummy import Pool
import cv2
import os
import glob
import numpy as np
from numpy import genfromtxt
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *


K.set_image_data_format('channels_first')

model = faceRecoModel(input_shape=(3,96,96))

def triplet_loss(y_true, y_pred, alpha=0.3):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1) 

    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)

    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss

model.compile(optimizer='adam', loss= triplet_loss, metrics=['accuracy'])


Image Embeddings from pre-trained FaceNet mode
when given a picture of a face, will extract high-quality features from it and predict a 128-element vector representation of these features, called a face embedding.

Standard Deviation, .std() is measured as the spread of data distribution in the given data set.



Preparing Dataset - Mapping person’s name to its embedding


Keep in mind that all the images that are fed to network must have 96*96 pixel images.

We are using,
**fr_utils** as a functions to feed images to the network and getting the encoding of images.




For importing python(.py) files:

import sys

sys.path.append('/content/drive/My Drive/')

import fr_utils

In [0]:
face_database = {}		# Dictionary having image name and corresponding embeddings of the image

path = "..path_to_your_folder_where_images_are_stored.."

for image in os.listdir(path):

		print('1st: ',os.path.join(path,image))
	
		identity = os.path.splitext(os.path.basename(image))[0]
		print('splitext: ',identity) 
	
		face_database[identity] = img_path_to_encoding(os.path.join(path,image), model)

print(face_database)

Facial Recognition

As we have our face database ready to use.
Here we will compute L2 distance  between target image and the avialable database.

For more about **norm** check this: https://hadrienj.github.io/posts/Deep-Learning-Book-Series-2.5-Norms/

Using only face instead of whole image for recognition problems help in better way. 
So, we will use OpenCV. 

In [0]:
import cv2
import pnslib
import sys

path = "..path_to_image.../img.jpg"
image = cv2.imread(path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#haarcascade_frontalface_default.xml needs to be downloaded
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor= 1.3,
    minNeighbors= 3,
    minSize=(30, 30)
)
print("[INFO] Found {0} Faces.".format(len(faces)))

for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    roi_img = image[y:y + h, x:x + w]
    print("[INFO] Object found. Saving ...")
    cv2.imwrite(str(w) + str(h) + '_IMG.jpg', roi_img)

status = cv2.imwrite('..path_where_you_want_to_save_roi_image../img.jpg', roi_img)

print("[INFO] Image faces_detected.jpg written to filesystem: ", status)

Identify

In [0]:
from numpy import linalg as LA

def recog(target, database, model): 
    # target is the image that we want to recognize
    encoding = img_path_to_encoding(path, model)

    min_dist =  1.0
    identity = None

    for (name, db_enc) in database.items():
        
        dist = LA.norm(db_enc - encoding)
        
        print("Distance for %s is %s " %(name, dist))

        if dist < min_dist:
            min_dist = dist
            identity = name

    if min_dist > 0.07:                                 
        return "Unknown"

    else:
        return (identity)


Here 0.07 came from trail and error for this dataset.
You have to find it for your face.

In [0]:
path = "..path_to_image_which_is_to_be_verify../img.jpg"
recog(path, face_database, model)
